# **Counting Algorithms**

## **Import Libraries**

In [1]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

## **Install Ultralytics**

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

## **Connect to Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Prepare Folder Path YOLOv8 Data**

In [4]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

## **Creating the Training and Test Set**

In [5]:
'''
Split the dataset into train and test and creates the train.txt and test.txt with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")

    files = list(set([name[:-4] for name in os.listdir(path)]))

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)]))
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")

    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    print("------ TASK COMPLETED -------")

train_test_split('/content/drive/MyDrive/yolov8/data/')

------ PROCESS STARTED -------
--- This folder has a total number of 220 images---


  0%|          | 0/176 [00:00<?, ?it/s]

------ Training data created with 80% split 176 images -------


  0%|          | 0/44 [00:00<?, ?it/s]

------ Testing data created with a total of 44 images ----------
------ TASK COMPLETED -------


## **Check Ultralytics**

In [6]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.7/78.2 GB disk)


## **Training the YOLOv8**

In [7]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=24 imgsz=1600 batch=8 project=/content/drive/MyDrive/yolov8/training_results

100% 21.5M/21.5M [00:00<00:00, 414MB/s]
Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=24, time=None, patience=100, batch=8, imgsz=1600, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

## **Testing the YOLOv8**

In [8]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/train/weights/best.pt conf=0.25 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs

image 1/24 /content/drive/MyDrive/yolov8/test_images/IMG_1030.jpg: 1088x1600 2 Object 1s, 1 Object 3, 1 Object 4, 70.0ms
image 2/24 /content/drive/MyDrive/yolov8/test_images/IMG_1031.jpg: 1088x1600 1 Object 1, 1 Object 3, 1 Object 4, 54.5ms
image 3/24 /content/drive/MyDrive/yolov8/test_images/IMG_3188.jpg: 1088x1600 2 Object 1s, 2 Object 3s, 54.7ms
image 4/24 /content/drive/MyDrive/yolov8/test_images/IMG_3189.jpg: 1088x1600 2 Object 1s, 2 Object 3s, 2 Object 4s, 54.6ms
image 5/24 /content/drive/MyDrive/yolov8/test_images/IMG_3190.jpg: 1088x1600 2 Object 1s, 1 Object 3, 54.6ms
image 6/24 /content/drive/MyDrive/yolov8/test_images/IMG_3191.jpg: 1088x1600 2 Object 1s, 1 Object 3, 54.5ms
image 7/24 /content/drive/MyDrive/yolov8/test_images/IMG_3192.jpg: 1088x1600 2 Object 1s, 1 Object 4, 54.6ms
image 8/24 /content/drive/MyDr

## **Copy the Prediction to GDrive**

In [9]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output